In [1]:
from pyspark.sql import SparkSession
from datetime import datetime
from pyspark.sql import Row
import time

spark = SparkSession.builder \
    .appName("ETLProcess") \
    .enableHiveSupport() \
    .getOrCreate()



In [2]:
def load_table_with_logging(table_name: str, csv_path: str, current_max_id: int):
    start_time = datetime.now()
    job_name = f"load_{table_name}"
    status = "SUCCESS"

    try:
        df = spark.read.csv(csv_path, sep=';', header=True, inferSchema=True)
        df.write.mode("overwrite").saveAsTable(table_name)
    except Exception as e:
        status = "FAILURE"
        print(f"Error loading {table_name}: {e}")

    time.sleep(5)

    end_time = datetime.now()
    new_id = current_max_id + 1

    log_row = Row(
        id=new_id,
        job_name=job_name,
        start_time=start_time,
        end_time=end_time,
        status=status
    )
    log_df = spark.createDataFrame([log_row])
    log_df.write.mode("append").insertInto("DS.LOGS")

    return new_id

In [3]:
# ==== Получение текущего максимального ID ====
try:
    current_max = spark.table("DS.LOGS").agg(spark_max("id")).collect()[0][0]
    current_max = current_max if current_max is not None else 0
except:
    current_max = 0
print(current_max)

0


In [4]:
tables_to_load = [
    ("DS.FT_BALANCE_F", "/data/ft_balance_f.csv"),
    ("DS.FT_POSTING_F", "/data/ft_posting_f.csv"),
    ("DS.MD_ACCOUNT_D", "/data/md_account_d.csv"),
    ("DS.MD_CURRENCY_D", "/data/md_currency_d.csv"),
    ("DS.MD_EXCHANGE_RATE_D", "/data/md_exchange_rate_d.csv"),
    ("DS.MD_LEDGER_ACCOUNT_S", "/data/md_ledger_account_s.csv"),
]

In [5]:
for job_name, csv_path in tables_to_load:
    current_max = load_table_with_logging(job_name, csv_path, current_max)

In [6]:
spark.sql("SELECT * FROM DS.LOGS").show(truncate=False)

+---+---------------------------+--------------------------+--------------------------+-------+
|id |job_name                   |start_time                |end_time                  |status |
+---+---------------------------+--------------------------+--------------------------+-------+
|3  |load_DS.MD_ACCOUNT_D       |2025-07-02 18:45:46.823517|2025-07-02 18:45:53.158756|SUCCESS|
|1  |load_DS.FT_BALANCE_F       |2025-07-02 18:45:27.423546|2025-07-02 18:45:37.145171|SUCCESS|
|1  |load_ft_balance_f.csv      |2025-07-02 17:54:00.497638|2025-07-02 17:54:06.09876 |SUCCESS|
|5  |load_DS.MD_EXCHANGE_RATE_D |2025-07-02 18:46:01.195983|2025-07-02 18:46:07.217475|SUCCESS|
|6  |load_DS.MD_LEDGER_ACCOUNT_S|2025-07-02 18:46:07.781244|2025-07-02 18:46:13.775723|SUCCESS|
|2  |load_DS.FT_POSTING_F       |2025-07-02 18:45:39.34216 |2025-07-02 18:45:45.694027|SUCCESS|
|4  |load_DS.MD_CURRENCY_D      |2025-07-02 18:45:54.173007|2025-07-02 18:46:00.196014|SUCCESS|
+---+---------------------------+-------